In [ ]:
import operator
import random
import time
import math
import multiprocessing as mp
from deap import base, creator, tools

#def cost_func(part):
#    x1, x2 = part[0], part[1]
#    return ((x1**2+x2**2)**0.25)*((math.sin(50*(x1**2+x2**2)**0.1))**2 +1.0),
s = sopt.fsolve(syst.ini_problem,x0 )
def cost_func(part):
    T_pss_1, T_pss_2 = part[0], part[1]
    x0 = np.ones((syst.N_x+syst.N_y,1))
    
    syst.struct[0].x[:,0] = s[0:syst.N_x]
    syst.struct[0].y[:,0] = s[syst.N_x:]
    syst.struct[0].T_pss_1 = T_pss_1                
    syst.struct[0].T_pss_2 = T_pss_2                
    T,X,Y = solver(syst.struct)
    cost = np.sum((Y[:,0] - (Y[:,4]- Y[:,1]))**2)
    #    a = ((x1**2+x2**2)**0.25)*((math.sin(50*(x1**2+x2**2)**0.1))**2 +1.0)
    return cost,

def generate(size, pmin, pmax, smin, smax):
    part = creator.Particle(random.uniform(pmin, pmax) for _ in range(size)) 
    part.speed = [random.uniform(smin, smax) for _ in range(size)]
    part.smin = smin
    part.smax = smax
    return part

def updateParticle(best, part, phi1, phi2):
    u1 = (random.uniform(0, phi1) for _ in range(len(part)))
    u2 = (random.uniform(0, phi2) for _ in range(len(part)))
    v_u1 = map(operator.mul, u1, map(operator.sub, part.best, part))
    v_u2 = map(operator.mul, u2, map(operator.sub, best, part))
    part.speed = list(map(operator.add, part.speed, map(operator.add, v_u1, v_u2)))
    for i, speed in enumerate(part.speed):
        if speed < part.smin:
            part.speed[i] = part.smin
        elif speed > part.smax:
            part.speed[i] = part.smax
    part[:] = list(map(operator.add, part, part.speed))
    return part

creator.create("FitnessMax", base.Fitness, weights=(-1.0,))
creator.create("Particle", list, fitness=creator.FitnessMax, speed=list, smin=None, smax=None, best=None)

toolbox = base.Toolbox()
#toolbox.register("particle", generate, size=2, pmin=-10, pmax=10, smin=-2, smax=2)
toolbox.register("particle", generate, size=2, pmin=0.001, pmax=10, smin=0.001, smax=10)
toolbox.register("population", tools.initRepeat, list, toolbox.particle)
toolbox.register("update", updateParticle, phi1=1.0, phi2=1.0)
toolbox.register("evaluate", cost_func)

def pso(pop,toolbox,maxmov):
    MOVES = maxmov
    best = None
    valor_best = None
    i = 0
    while i < MOVES:
        print('iteracion', i)
        fitnesses = toolbox.map(toolbox.evaluate,pop)
        for part, fit in zip(pop, fitnesses):
             part.fitness.values = fit
        for part in pop:
            if not part.best or part.best.fitness < part.fitness:
                part.best = creator.Particle(part)
                part.best.fitness.values = part.fitness.values
            if not best or best.fitness < part.fitness:
                best = creator.Particle(part)
                best.fitness.values = part.fitness.values
        valor_best1 = best.fitness.values
        if valor_best == valor_best1:
            i += 1
        else:
            valor_best = valor_best1
            i = 0
        for part in pop:
            toolbox.update(best, part)
    return best, best.fitness